<a href="https://colab.research.google.com/github/jtapiav/github-slideshow/blob/master/Extending_Kernighan_Lin_Partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [ ]:
import random

#Original Kernighan-Lin Partition

## Graph

In [ ]:
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    cost = 0
    for edge in graph.edges:
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            cost += 1
    return cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2):
    best_cost = calculate_cost( graph, partition1, partition2 )
    best_move = None
    for ci in partition1:
        for cj in partition2:
            current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
            if current_cost < best_cost:
                best_cost = current_cost
                best_move = (ci, cj)
    return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 50

    for iteration in range(max_iterations):
        next_move = get_next_move(graph, current_partition1, current_partition2)
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
        return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C15', 'C4', 'C7', 'C10', 'C2', 'C1', 'C13', 'C16', 'C6', 'C14', 'C9', 'C8', 'C3', 'C11', 'C5', 'C12'}
Start Partitions Cost: 10
 Partition 1: {'C13', 'C4', 'C7', 'C16', 'C6', 'C8', 'C10', 'C5'}
 Partition 2: {'C15', 'C14', 'C9', 'C11', 'C3', 'C2', 'C1', 'C12'}
Final Partitions Cost: 4
 Partition 1: {'C4', 'C7', 'C6', 'C9', 'C8', 'C3', 'C10', 'C5'}
 Partition 2: {'C15', 'C16', 'C14', 'C11', 'C2', 'C1', 'C13', 'C12'}
    Level 1:
    Complete Set: {'C4', 'C7', 'C6', 'C9', 'C8', 'C3', 'C10', 'C5'}
    Start Partitions Cost: 4
     Partition 1: {'C4', 'C7', 'C8', 'C6'}
     Partition 2: {'C9', 'C3', 'C5', 'C10'}
    Final Partitions Cost: 4
     Partition 1: {'C4', 'C7', 'C8', 'C6'}
     Partition 2: {'C9', 'C3', 'C5', 'C10'}
        Level 2:
        Complete Set: {'C4', 'C7', 'C8', 'C6'}
        Start Partitions Cost: 2
         Partition 1: {'C4', 'C8'}
         Partition 2: {'C7', 'C6'}
        Final Partitions Cost: 0
         Partition 1: {'C7', 'C8'}
        

# Weighted Kernighan-Lin Partition

## Graph

In [ ]:
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges
        self.weights = {edge: random.randint(1, 10) for edge in edges}

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    cost = 0
    for edge, weight in graph.weights.items():
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            cost += weight
    return cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2):
    best_cost = calculate_cost( graph, partition1, partition2 )
    best_move = None
    for ci in partition1:
        for cj in partition2:
            current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
            if current_cost < best_cost:
                best_cost = current_cost
                best_move = (ci, cj)
    return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 50

    for iteration in range(max_iterations):
        next_move = get_next_move(graph, current_partition1, current_partition2)
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
        return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C15', 'C4', 'C7', 'C10', 'C2', 'C1', 'C13', 'C16', 'C6', 'C14', 'C9', 'C8', 'C3', 'C11', 'C5', 'C12'}
Start Partitions Cost: 54
 Partition 1: {'C15', 'C4', 'C6', 'C11', 'C8', 'C3', 'C1', 'C13'}
 Partition 2: {'C7', 'C16', 'C14', 'C9', 'C10', 'C2', 'C5', 'C12'}
Final Partitions Cost: 5
 Partition 1: {'C13', 'C4', 'C6', 'C14', 'C3', 'C2', 'C1', 'C5'}
 Partition 2: {'C15', 'C7', 'C16', 'C9', 'C11', 'C8', 'C10', 'C12'}
    Level 1:
    Complete Set: {'C13', 'C4', 'C6', 'C14', 'C3', 'C2', 'C1', 'C5'}
    Start Partitions Cost: 29
     Partition 1: {'C2', 'C5', 'C3', 'C14'}
     Partition 2: {'C4', 'C1', 'C6', 'C13'}
    Final Partitions Cost: 13
     Partition 1: {'C2', 'C1', 'C3', 'C5'}
     Partition 2: {'C13', 'C4', 'C6', 'C14'}
        Level 2:
        Complete Set: {'C2', 'C1', 'C3', 'C5'}
        Start Partitions Cost: 6
         Partition 1: {'C3', 'C5'}
         Partition 2: {'C2', 'C1'}
        Final Partitions Cost: 6
         Partition 1: {'C3', 'C5'}
   

# Weighted Internal-External Kernighan-Lin Partition

## Graph

In [ ]:
import random
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges
        self.weights = {edge: random.randint(1, 10) for edge in edges}

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    external_cost = 0
    internal_cost = 0

    for edge, weight in graph.weights.items():
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            external_cost += weight
        elif (edge[0] in partition1 and edge[1] in partition1) or (edge[0] in partition2 and edge[1] in partition2):
            internal_cost += weight

    return external_cost - internal_cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2, locked_vertices):
  best_cost = calculate_cost( graph, partition1, partition2 )
  best_move = None
  for ci in partition1:
    if ci in locked_vertices:
      continue
    for cj in partition2:
      if cj in locked_vertices:
        continue
      current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
      if current_cost < best_cost:
        best_cost = current_cost
        best_move = (ci, cj)
  return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 5

    for iteration in range(max_iterations):
      locked_vertices = set()
      print( iteration )
      while( len(current_partition1) > 1 ):
        next_move = get_next_move(graph, current_partition1, current_partition2, locked_vertices)
        print( next_move )
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)
        locked_vertices.add(ci)
        locked_vertices.add(cj)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
      return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C15', 'C4', 'C7', 'C10', 'C2', 'C1', 'C13', 'C16', 'C6', 'C14', 'C9', 'C8', 'C3', 'C11', 'C5', 'C12'}
Start Partitions Cost: 13
 Partition 1: {'C15', 'C4', 'C14', 'C5', 'C11', 'C1', 'C13', 'C12'}
 Partition 2: {'C7', 'C16', 'C6', 'C9', 'C8', 'C3', 'C10', 'C2'}
0
('C5', 'C16')
('C4', 'C2')
None
1
None
2
None
3
None
4
None
Final Partitions Cost: -63
 Partition 1: {'C15', 'C16', 'C14', 'C11', 'C2', 'C1', 'C13', 'C12'}
 Partition 2: {'C4', 'C7', 'C6', 'C9', 'C8', 'C3', 'C10', 'C5'}
    Level 1:
    Complete Set: {'C15', 'C16', 'C14', 'C11', 'C2', 'C1', 'C13', 'C12'}
    Start Partitions Cost: -6
     Partition 1: {'C15', 'C11', 'C13', 'C12'}
     Partition 2: {'C2', 'C1', 'C16', 'C14'}
0
('C15', 'C14')
None
1
None
2
None
3
None
4
None
    Final Partitions Cost: -20
     Partition 1: {'C12', 'C11', 'C13', 'C14'}
     Partition 2: {'C15', 'C1', 'C2', 'C16'}
        Level 2:
        Complete Set: {'C12', 'C11', 'C13', 'C14'}
        Start Partitions Cost: 19
         

# Weight Internal-External Kernighan-Lin Partition - Cost Matrix

## Graph

In [ ]:
import random
class CoreGraph:
    def __init__(self, cores, edges, weight_matrix):
        self.cores = cores
        self.edges = edges
        self.weight_matrix = weight_matrix

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    external_cost = 0
    internal_cost = 0

    cores_list = list(graph.cores)  # Convertimos el conjunto de cores a lista
    cores_internal_cost_list = []
    cores_external_cost_list = []
    cores_reduction_cost_list = []

    for ci in partition1:
      print( ci )

    for edge in graph.edges:
        i, j = cores_list.index(edge[0]), cores_list.index(edge[1])
        weight = graph.weight_matrix[i][j]

        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            external_cost += weight
        elif (edge[0] in partition1 and edge[1] in partition1) or (edge[0] in partition2 and edge[1] in partition2):
            internal_cost += weight

    return external_cost - internal_cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2, locked_vertices):
  best_cost = calculate_cost( graph, partition1, partition2 )
  best_move = None
  for ci in partition1:
    if ci in locked_vertices:
      continue
    for cj in partition2:
      if cj in locked_vertices:
        continue
      current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
      if current_cost < best_cost:
        best_cost = current_cost
        best_move = (ci, cj)
  return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 5

    for iteration in range(max_iterations):
      locked_vertices = set()
      print( iteration )
      while( len(current_partition1) > 1 ):
        next_move = get_next_move(graph, current_partition1, current_partition2, locked_vertices)
        print( next_move )
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)
        locked_vertices.add(ci)
        locked_vertices.add(cj)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
      return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6'}

edges = {('C1', 'C2'), ('C1', 'C3'), ('C1', 'C4'), ('C1', 'C5'),
         ('C2', 'C1'), ('C2', 'C3'), ('C2', 'C4'), ('C2', 'C5'),
         ('C3', 'C1'), ('C3', 'C2'), ('C3', 'C4'), ('C3', 'C5'),
         ('C4', 'C1'), ('C4', 'C2'), ('C4', 'C3'), ('C4', 'C5'),
         ('C5', 'C1'), ('C5', 'C2'), ('C5', 'C3'), ('C5', 'C4')}

weight_matrix = [
    [0, 1, 2, 3, 2, 4],
    [1, 0, 1, 4, 2, 1],
    [2, 1, 0, 3, 2, 1],
    [3, 4, 3, 0, 4, 3],
    [2, 2, 2, 4, 0, 2],
    [4, 1, 1, 3, 2, 0]
]



graph = CoreGraph(cores, edges, weight_matrix)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C5', 'C3', 'C2', 'C4', 'C1', 'C6'}
C4
C3
C6
Start Partitions Cost: 8
 Partition 1: {'C4', 'C3', 'C6'}
 Partition 2: {'C2', 'C1', 'C5'}
0
C4
C3
C6
C2
C6
C3
C6
C1
C3
C6
C5
C3
C2
C4
C6
C4
C1
C6
C4
C5
C6
C2
C4
C3
C4
C1
C3
C4
C5
C3
('C6', 'C1')
C4
C3
C1
C4
C3
C6
C4
C3
C1
C2
C1
C3
C1
C3
C5
C2
C4
C1
C4
C1
C5
None
1
C4
C3
C1
C2
C1
C3
C6
C1
C3
C1
C3
C5
C2
C4
C1
C4
C1
C6
C4
C1
C5
C2
C4
C3
C4
C6
C3
C4
C5
C3
None
2
C4
C3
C1
C2
C1
C3
C6
C1
C3
C1
C3
C5
C2
C4
C1
C4
C1
C6
C4
C1
C5
C2
C4
C3
C4
C6
C3
C4
C5
C3
None
3
C4
C3
C1
C2
C1
C3
C6
C1
C3
C1
C3
C5
C2
C4
C1
C4
C1
C6
C4
C1
C5
C2
C4
C3
C4
C6
C3
C4
C5
C3
None
4
C4
C3
C1
C2
C1
C3
C6
C1
C3
C1
C3
C5
C2
C4
C1
C4
C1
C6
C4
C1
C5
C2
C4
C3
C4
C6
C3
C4
C5
C3
None
C4
C1
C3
Final Partitions Cost: 0
 Partition 1: {'C4', 'C1', 'C3'}
 Partition 2: {'C2', 'C5', 'C6'}


ValueError: ignored

In [88]:
import numpy as np

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):
  internal_adj_matrix = np.ones((num_nodes,num_nodes))
  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, external_adj_matrix, num_nodes ):
  external_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, internal_adj_matrix, num_nodes ):
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector(  partition1, partition2, num_nodes ):

  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)


  external_cost_matrix = create_external_cost_matrix( cost_matrix, external_adj_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, internal_adj_matrix, num_nodes )
  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[i][j] - internal_cost_matrix[i][j]

  return np.dot([1,1,1,1,1,1],difference_cost_matrix)


def calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  max_element = -10
  i_max_element = 0
  j_max_element = 0
  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if max_element < gain_cost_matrix[i][j] and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1:
        max_element = gain_cost_matrix[i][j]
        i_max_element = i
        j_max_element = j

  print( i_max_element, j_max_element, max_element )
  for ci in partition1:
    for cj in partition2:
      if ci == i_max_element and cj == j_max_element:
        partition1.remove(ci)
        partition2.remove(cj)
        partition1.add(cj)
        partition2.add(ci)

  return partition1, partition2


In [89]:
# Ejemplo de uso
if __name__ == "__main__":
  partition1 = {0, 1, 2}
  partition2 = {3, 4, 5}
  num_nodes = 6

  cost_matrix = [ [0,1,2,3,2,4],
                  [1,0,1,4,2,1],
                  [2,1,0,3,2,1],
                  [3,4,3,0,4,3],
                  [2,2,2,4,0,2],
                  [4,1,1,3,2,0] ]


  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)


  external_cost_matrix = create_external_cost_matrix( cost_matrix, external_adj_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, internal_adj_matrix, num_nodes )

  difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )

  gain_cost_matrix        =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )




  print("*-"*20)
  for row in external_adj_matrix:
    print(row)

  print("-"*40)
  for row in internal_adj_matrix:
    print(row)

  print("*-"*20)



  for row in  internal_cost_matrix:
    print(row, np.sum(row))

  print("-"*40)



  for row in external_cost_matrix:
    print(row, np.sum(row))

  print("-"*40)


  print( difference_cost_vector )

  print("*-"*20)

  for row in  gain_cost_matrix:
    print(row, np.sum(row))


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
[0. 0. 0. 1. 1. 1.]
[0. 0. 0. 1. 1. 1.]
[0. 0. 0. 1. 1. 1.]
[1. 1. 1. 0. 0. 0.]
[1. 1. 1. 0. 0. 0.]
[1. 1. 1. 0. 0. 0.]
----------------------------------------
[1. 1. 1. 0. 0. 0.]
[1. 1. 1. 0. 0. 0.]
[1. 1. 1. 0. 0. 0.]
[0. 0. 0. 1. 1. 1.]
[0. 0. 0. 1. 1. 1.]
[0. 0. 0. 1. 1. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
[0. 1. 2. 0. 0. 0.] 3.0
[1. 0. 1. 0. 0. 0.] 2.0
[2. 1. 0. 0. 0. 0.] 3.0
[0. 0. 0. 0. 4. 3.] 7.0
[0. 0. 0. 4. 0. 2.] 6.0
[0. 0. 0. 3. 2. 0.] 5.0
----------------------------------------
[0. 0. 0. 3. 2. 4.] 9.0
[0. 0. 0. 4. 2. 1.] 7.0
[0. 0. 0. 3. 2. 1.] 6.0
[3. 4. 3. 0. 0. 0.] 10.0
[2. 2. 2. 0. 0. 0.] 6.0
[4. 1. 1. 0. 0. 0.] 6.0
----------------------------------------
[6. 5. 3. 3. 0. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
[ 0  0  0  3  2 -1] 4
[0 0 0 0 1 4] 5
[ 0  0  0  0 -1  2] 1
[3 0 0 0 0 0] 3
[ 2  1 -1  0  0  0] 2
[-1  4  2  0  0  0] 5


In [90]:
  partition1, partition2 = swap_cores_partition( partition1, partition2, gain_cost_matrix, [1,1,1,1,1,1] )

  print( "Partition1:", partition1 )
  print( "Partition2:", partition2 )

1 5 4
Partition1: {0, 2, 5}
Partition2: {1, 3, 4}


In [91]:
  def update_difference_cost_vector(  difference_cost_vector,   ):

    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        if i == 1 and j!=5 or i == 5 and j!=1:
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]


    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]



    difference_cost_vector          = np.multiply([1,0,1,1,1,0], difference_cost_vector)
    difference_cost_internal_vector = np.dot([0,1,0,0,0,1], difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot([0,1,0,0,0,1], difference_cost_external_matrix)
    difference_cost_prime_vector = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector





    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("-"*40)

    print("\n")

    print("*-"*20)
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    return difference_cost_external_matrix, difference_cost_internal_matrix, difference_cost_prime_vector

  difference_cost_external_matrix, difference_cost_internal_matrix, difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector )

  print("*-"*20)
  print( np.dot([1,1,1,1,1,1], external_cost_matrix-internal_cost_matrix)  )
  print("-"*40)
  print( np.dot([0,1,0,0,0,1], difference_cost_external_matrix)  )
  print("-"*40)
  print( np.dot([0,1,0,0,0,1], difference_cost_internal_matrix)  )
  print("*-"*20)
  print( np.dot([1,0,1,1,1,0], ( np.dot([1,1,1,1,1,1], external_cost_matrix-internal_cost_matrix)  ) + 2*( np.dot([0,1,0,0,0,1], difference_cost_internal_matrix)  ) - 2*( np.dot([0,1,0,0,0,1], difference_cost_external_matrix)  ) )  )
  print("*-"*20)
  print( np.multiply( [1,0,1,1,1,0], calculate_gain_cost_matrix( np.multiply([1,0,1,1,1,0], ( np.dot([1,1,1,1,1,1], external_cost_matrix-internal_cost_matrix)  ) + 2*( np.dot([0,1,0,0,0,1], difference_cost_internal_matrix)  ) - 2*( np.dot([0,1,0,0,0,1], difference_cost_external_matrix)  ) ), create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )  ) )

  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
  print(   gain_cost_matrix  )

  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector, create_external_adj_matrix( partition1, partition2, num_nodes ),   cost_matrix  )
  for row in  gain_cost_matrix:
    print(row, np.sum(row))



*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 1. 2. 0. 0. 0.] 3.0
[1. 0. 1. 0. 0. 0.] 2.0
[2. 1. 0. 0. 0. 0.] 3.0
[0. 0. 0. 0. 4. 3.] 7.0
[0. 0. 0. 4. 0. 2.] 6.0
[0. 0. 0. 3. 2. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[1. 0. 1. 3. 2. 0.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 0. 0. 0. 0. 0.] 0.0
[0. 0. 0. 4. 2. 0.] 6.0
[0. 0. 0. 0. 0. 0.] 0.0
[0. 0. 0. 0. 0. 0.] 0.0
[0. 0. 0. 0. 0. 0.] 0.0
[4. 0. 1. 0. 0. 0.] 5.0
----------------------------------------
Difference Cost External Vector
[4. 0. 1. 4. 2. 0.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


Difference Cost Vector
[6. 0. 3. 3. 0. 0.]
----------------------------------------
Difference Cost Updated Vector
[0. 0. 3. 1. 0. 0.]
----------------------------------------


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
[0. 0. 0. 0. 0. 0.] 0.0
[0. 0. 0. 4. 2. 0.] 6.0
[0. 0. 0. 0. 0. 0.] 0.0
[0.

In [92]:
  print(np.multiply([[1],[0],[1],[1],[1],[0]],np.multiply([1,0,1,1,1,0], gain_cost_matrix)))
  partition1, partition2 = swap_cores_partition( partition1, partition2, np.multiply([[1],[0],[1],[1],[1],[0]], np.multiply([1,0,1,1,1,0], gain_cost_matrix) ) , [1,0,1,1,1,0] )
  partition1, partition2 =  swap_cores_partition( partition1, partition1, gain_cost_matrix, [1,0,1,1,1,0]  )


  print( "Partition1:", partition1 )
  print( "Partition2:", partition2 )

[[ 0  0  0 -5 -4  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0 -2 -1  0]
 [-5  0 -2  0  0  0]
 [-4  0 -1  0  0  0]
 [ 0  0  0  0  0  0]]
2 4 -1
2 4 -1
Partition1: {0, 4, 5}
Partition2: {0, 4, 5}


In [49]:
updated_external_cost_matrix = np.multiply( [1,0,1,1,1,0] ,external_cost_matrix )
print( updated_external_cost_matrix  )


[[0. 0. 0. 3. 2. 0.]
 [0. 0. 0. 4. 2. 0.]
 [0. 0. 0. 3. 2. 0.]
 [3. 0. 3. 0. 0. 0.]
 [2. 0. 2. 0. 0. 0.]
 [4. 0. 1. 0. 0. 0.]]


NameError: ignored